In [104]:
import jax

import jax.numpy as jnp
import numpy as np
from typing import NamedTuple,Optional
import matplotlib.pyplot as plt
np.random.seed(0)
S = 10 #state_size
A = 4  #action_size
S_array = np.arange(S) #state_set
A_array = np.arange(A) #action_set
gamma = 0.9

rew = np.random.rand(S) #報酬

P = np.random.rand(S, S) #遷移確率
P = P / np.sum(P,axis=-1,keepdims=True)
P = P.reshape(S,S)
np.testing.assert_almost_equal(P.sum(axis=-1),1) #確率1になっているか確認

# MDPの定義

class MDP(NamedTuple):
    S_array : np.array
    A_array : np.array
    gamma : float
    rew : np.array
    P : np.array
    horizon : int

    @property
    def S(self) -> int:
        return len(self.S_array)

    @property
    def A(self) -> int:
        return len(self.A_array)

horizon = int(1 / (1 - gamma))
mdp = MDP(S_array,A_array,gamma,rew,P,horizon)
print(mdp.rew.shape)
print(mdp.P.shape)

(10,)
(10, 10)


In [131]:
import numpy as np

def quantile_dynamic_programming(theta,  MDP, gamma):
    X = len(theta)
    m = len(theta[0])

    theta_prime = np.zeros_like(theta)

    for x in range(MDP.S):
        probabilities = []
        outcomes = []
        for x_prime in range(MDP.S):
            for a in range(MDP.A):
                for j in range(m):
                    theta[x] = MDP.rew[x] + gamma * theta[x_prime, j]
                    probability = MDP.P[x,x_prime] / m
                    outcomes.append(theta[x])
                    probabilities.append(probability)

        # Sort targets ascending according to outcomes
        # print(targets)
        # targets = np.array(targets, dtype=[('outcome', float), ('probability', float)])
        # targets = targets.reshape(MDP.S, m)
        # print(targets[0])
        # print('-----')
        # targets.sort(order='outcome')
        outcomes = np.array(outcomes)
        probabilities = np.array(probabilities)

        # Sort targets based on outcomes
        sorted_indices = np.argsort(outcomes)
        outcomes = outcomes[sorted_indices]
        probabilities = probabilities[sorted_indices]

        cumulative_probability = 0
        for i in range(1, m + 1):
            target_index = 0
            # print(target_index < len(probabilities) and cumulative_probability < (2 * i - 1) / (2 * m))
            while target_index < len(probabilities) and cumulative_probability < (2 * i - 1) / (2 * m):
                cumulative_probability += probabilities[target_index][0]
                # print(cumulative_probability)
                target_index += 1
                # print('a')

            q_prime = outcomes[target_index - 1]
            # print(q_prime[0][2])
            theta_prime[x, i - 1] = q_prime[x][i-1]

    return theta_prime

# Example usage
# Set initial parameters
m = 100 # Number of quantiles
gamma = 0.9  # Discount factor

# Dummy quantile estimates
theta = np.random.rand(mdp.S, m)

# Dummy transition and reward probabilities
# transition_probs = {x_prime: {r: 1 / m for r in range(-1, 2)} for x_prime in range(X)}

# Dummy interpolation parameters
# interpolation_params = np.random.rand(X, m)

# Run the quantile dynamic programming algorithm
result = quantile_dynamic_programming(theta,mdp,gamma)

print("Updated quantile estimates:", result.shape)

print(f'original value: {result.mean(axis=-1)}')


Updated quantile estimates: (10, 100)
original value: [0.83751738 1.00389324 0.89146725 0.83358706 0.71235867 0.93459799
 0.72629109 1.18047687 1.25236663 3.83441519]


In [132]:
def V_Iteration(mdp: MDP,) -> np.array:
    S,A = mdp.S,mdp.A
    Q = np.zeros((S,A))
    policy = np.zeros((S,A))
    for s in range(S):
        for a in range(A):
            for s_dash in range(S):
                Q[s,a] += mdp.rew[s,a] + mdp.gamma * mdp.P[s,a,s_dash] * np.max(Q[s_dash,:])
    V = np.max(Q,axis=-1)
    return V

In [133]:
V = V_Iteration(mdp)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [134]:
V - result.mean(axis=-1)

array([ 77.57327773, 102.12569468, 120.96130182, 108.81023858,
       122.06164951, 159.31651006, 145.68946077, 153.95669538,
       173.9101292 , 207.2481191 ])

In [99]:
V

array([ 78.41079511, 103.12958792, 121.85276907, 109.64382563,
       122.77400818, 160.25110804, 146.41575185, 155.13717225,
       175.16249583, 211.08253429])